In [1]:
import pandas as pd
import numpy as np

In [15]:
from ddf_utils.str import format_float_digits

In [2]:
!ls -1 ../source/who

rated_0_14.csv
rated_15_29.csv
rated_30_44.csv
rated_45_59.csv
rated_60+.csv
ratedall.csv


In [3]:
!ls -1 ../source/gbd

deaths_rate_014.csv
deaths_rate_1529.csv
deaths_rate_3044.csv
deaths_rate_4559.csv
deaths_rate_60plus.csv
deaths_rate_all_age.csv


In [4]:
# note: both sexes is 3 in GBD but 0 in WHO, need to change that first

In [6]:
col_who = ['geo', 'year', 'cause', 'sex', 'value']
col_gbd = ['geo', 'sex', 'cause', 'year', 'value']

In [7]:
def trend_bridge(left, right, bridge_length=10, bridge_point=1990):
    left_ = left.set_index('year').sort_index()['value']
    right_ = right.set_index('year').sort_index()['value']
    
    geo = right['geo'].values[0]
    cause = right['cause'].values[0]
    sex = right['sex'].values[0]
    
    if left_.index[0] >= bridge_point:
        return right
    
    if bridge_point not in left_.index:
        before = left_.loc[:bridge_point].index[-1]
        if left_.index[-1] < bridge_point:
            bridge_point = before
        else:
            after = left_.loc[bridge_point:].index[0]
            if bridge_point - before >= after - bridge_point:
                bridge_point = after
            else:
                bridge_point = before
    
    if bridge_point < 1990:
        # bridge_height = right_.loc[1990] - left_.loc[bridge_point]
        left_.loc[1990] = left_.loc[bridge_point]
        left_ = left_.sort_index()
        bridge_point = 1990
        # merged = pd.concat([left_.loc[:bridge_point], right_.loc[:]])
        
    bridge_height = right_.loc[bridge_point] - left_.loc[bridge_point]

    if len(left_.loc[:bridge_point]) < bridge_length:
        bridge_length = len(left_.loc[:bridge_point])

    left_ = left_.loc[:bridge_point]

    bridge_start = left_.index[-bridge_length]

    fraction = bridge_height / bridge_length

    for t in left_.loc[bridge_start:].index:
        left_.loc[t:] = left_.loc[t:] + fraction

    merged = pd.concat([left_.iloc[:-1], right_.loc[bridge_point:]])
        
    merged = merged.reset_index()
    merged['geo'] = geo
    merged['cause'] = cause
    merged['sex'] = sex
    
    merged = merged[['geo', 'sex', 'cause', 'year', 'value']]
    
    # import ipdb; ipdb.set_trace()
    
    return merged

In [10]:
def process(who_f, gbd_f):
    who = pd.read_csv(who_f)
    gbd = pd.read_csv(gbd_f)
    
    print(list(who.columns))
    print("=>")
    print(col_who)
    print(list(gbd.columns))
    print("=>")
    print(col_gbd)
    who.columns = col_who
    gbd.columns = col_gbd
    
    who = who[['geo', 'sex', 'cause', 'year', 'value']]
    gbd = gbd[['geo', 'sex', 'cause', 'year', 'value']]
    gbd['sex'] = gbd['sex'].replace(3, 0)
    
    all_geos = set(np.r_[who.geo.unique(), gbd.geo.unique()])
    
    
    to_concat = []

    for g in all_geos:
        for s in range(3):
            for c in ['homicide', 'suicide', 'traffic']:
                who_g = who[(who.geo == g) & (who.sex == s) & (who.cause == c)].copy()
                gbd_g = gbd[(gbd.geo == g) & (gbd.sex == s) & (gbd.cause == c)].copy()
                # print(who_g.dtypes['year'])
                # print(gbd_g.dtypes['year'])
                if who_g.empty and gbd_g.empty:
                    continue
                elif who_g.empty:
                    to_concat.append(gbd_g)
                elif gbd_g.empty:
                    to_concat.append(who_g)
                else:
                    to_concat.append(trend_bridge(who_g, gbd_g))
                    
    res = pd.concat(to_concat, ignore_index=True, sort=False)
    return res

In [7]:
# all_age

In [9]:
all_age = process('../source/who/ratedall.csv', '../source/gbd/deaths_rate_all_age.csv')

Index(['geo', 'year', 'cause', 'sex', 'ratedall'], dtype='object')
=>
['geo', 'year', 'cause', 'sex', 'value']
Index(['location', 'sex', 'cause', 'year', 'deaths_rate_all_age'], dtype='object')
=>
['geo', 'sex', 'cause', 'year', 'value']


In [16]:
all_age = all_age.sort_values(by=['geo', 'sex', 'cause', 'year'])

all_age['value'] = all_age['value'].map(format_float_digits)

all_age = all_age.rename(columns={'value': 'deaths_rate_all_ages'})

all_age.to_csv('../../ddf--datapoints--deaths_rate_all_ages--by--geo--sex--cause--year.csv', index=False)

In [17]:
all_age.head()

,geo,sex,cause,year,deaths_rate_all_ages
5886,afg,0,homicide,1990,20.41
5887,afg,0,homicide,1991,20.54
5888,afg,0,homicide,1992,20.76
5889,afg,0,homicide,1993,21.08
5890,afg,0,homicide,1994,21.46


In [11]:
# 014

In [32]:
rate014 = process('../source/who/rated_0_14.csv', '../source/gbd/deaths_rate_014.csv')

['geo', 'year', 'cause', 'sex', 'rated_0_14']
=>
['geo', 'year', 'cause', 'sex', 'value']
['location', 'sex', 'cause', 'year', 'deaths_rate']
=>
['geo', 'sex', 'cause', 'year', 'value']


In [33]:
rate014 = rate014.sort_values(by=['geo', 'sex', 'cause', 'year'])

rate014['value'] = rate014['value'].map(format_float_digits)

rate014 = rate014.rename(columns={'value': 'deaths_rate_0_14'})

rate014.to_csv('../../ddf--datapoints--deaths_rate_0_14--by--geo--sex--cause--year.csv', index=False)

In [20]:
rate014.head()

,geo,sex,cause,year,deaths_rate_0_14
4851,afg,0,homicide,1990,2.56621
4852,afg,0,homicide,1991,2.55601
4853,afg,0,homicide,1992,2.56598
4854,afg,0,homicide,1993,2.59601
4855,afg,0,homicide,1994,2.64272


In [ ]:
# 1529

In [21]:
rate1529 = process('../source/who/rated_15_29.csv', '../source/gbd/deaths_rate_1529.csv')

['geo', 'year', 'cause', 'sex', 'rated_15_29']
=>
['geo', 'year', 'cause', 'sex', 'value']
['location', 'sex', 'cause', 'year', 'deaths_rate']
=>
['geo', 'sex', 'cause', 'year', 'value']


In [22]:
rate1529 = rate1529.sort_values(by=['geo', 'sex', 'cause', 'year'])

rate1529['value'] = rate1529['value'].map(format_float_digits)

rate1529 = rate1529.rename(columns={'value': 'deaths_rate_15_29'})

rate1529.to_csv('../../ddf--datapoints--deaths_rate_15_29--by--geo--sex--cause--year.csv', index=False)

In [ ]:
#3044

In [24]:
rate3044 = process('../source/who/rated_30_44.csv', '../source/gbd/deaths_rate_3044.csv')

['geo', 'year', 'cause', 'sex', 'rated_30_44']
=>
['geo', 'year', 'cause', 'sex', 'value']
['location', 'sex', 'cause', 'year', 'deaths_rate']
=>
['geo', 'sex', 'cause', 'year', 'value']


In [25]:
rate3044 = rate3044.sort_values(by=['geo', 'sex', 'cause', 'year'])

rate3044['value'] = rate3044['value'].map(format_float_digits)

rate3044 = rate3044.rename(columns={'value': 'deaths_rate_30_44'})

rate3044.to_csv('../../ddf--datapoints--deaths_rate_30_44--by--geo--sex--cause--year.csv', index=False)

In [ ]:
#4559

In [26]:
rate4559 = process('../source/who/rated_45_59.csv', '../source/gbd/deaths_rate_4559.csv')

['geo', 'year', 'cause', 'sex', 'rated_45_59']
=>
['geo', 'year', 'cause', 'sex', 'value']
['location', 'sex', 'cause', 'year', 'deaths_rate']
=>
['geo', 'sex', 'cause', 'year', 'value']


In [27]:
rate4559 = rate4559.sort_values(by=['geo', 'sex', 'cause', 'year'])

rate4559['value'] = rate4559['value'].map(format_float_digits)

rate4559 = rate4559.rename(columns={'value': 'deaths_rate_45_59'})

rate4559.to_csv('../../ddf--datapoints--deaths_rate_45_59--by--geo--sex--cause--year.csv', index=False)

In [ ]:
# 60plus

In [28]:
rate60 = process('../source/who/rated_60+.csv', '../source/gbd/deaths_rate_60plus.csv')

['geo', 'year', 'cause', 'sex', 'rated_60+']
=>
['geo', 'year', 'cause', 'sex', 'value']
['location', 'sex', 'cause', 'year', 'deaths_rate']
=>
['geo', 'sex', 'cause', 'year', 'value']


In [29]:
rate60 = rate60.sort_values(by=['geo', 'sex', 'cause', 'year'])

rate60['value'] = rate60['value'].map(format_float_digits)

rate60 = rate60.rename(columns={'value': 'deaths_rate_60plus'})

rate60.to_csv('../../ddf--datapoints--deaths_rate_60plus--by--geo--sex--cause--year.csv', index=False)

In [30]:
# concepts

In [35]:
measures = ['deaths_rate_all_ages', 
            'deaths_rate_0_14', 
            'deaths_rate_15_29', 
            'deaths_rate_30_44', 
            'deaths_rate_45_59', 
            'deaths_rate_60plus'
           ]

cdf = pd.DataFrame(measures, columns=['concept'])

In [37]:
cdf['name'] = cdf['concept'].map(lambda x: x.replace('_', ' ').title())

In [38]:
cdf['concept_type'] = 'measure'

In [42]:
ddf = pd.DataFrame([['name', 'Name', 'string'],
                    ['geo', 'Geo', 'entity_domain'],
                    ['sex', 'Sex', 'entity_domain'],
                    ['cause', 'Cause', 'entity_domain'],
                    ['year', 'Year', 'time']], columns=['concept', 'name', 'concept_type'])

In [43]:
ddf

,concept,name,concept_type
0,name,Name,string
1,geo,Geo,entity_domain
2,sex,Sex,entity_domain
3,cause,Cause,entity_domain
4,year,Year,time


In [44]:
concepts_df = pd.concat([cdf, ddf], ignore_index=True)

In [46]:
concepts_df.to_csv('../../ddf--concepts.csv', index=False)

In [ ]:
# below are drafts

In [8]:
who = pd.read_csv('../source/who/ratedall.csv')
gbd = pd.read_csv('../source/gbd/deaths_rate_all_age.csv')

In [9]:
who.columns

Index(['geo', 'year', 'cause', 'sex', 'ratedall'], dtype='object')

In [10]:
gbd.columns

Index(['location', 'sex', 'cause', 'year', 'deaths_rate_all_age'], dtype='object')

In [11]:
who.columns = col_who
gbd.columns = col_gbd

In [12]:
who = who[['geo', 'sex', 'cause', 'year', 'value']]

In [13]:
gbd['sex'] = gbd['sex'].replace(3, 0)

In [14]:
who.shape

(22822, 5)

In [15]:
gbd.shape

(47142, 5)

In [16]:
22822 + 47142

69964

In [17]:
all_geos = set(np.r_[who.geo.unique(), gbd.geo.unique()])

In [41]:
to_concat = []

for g in all_geos:
    for s in range(3):
        for c in ['homicide', 'suicide', 'traffic']:
            who_g = who[(who.geo == g) & (who.sex == s) & (who.cause == c)].copy()
            gbd_g = gbd[(gbd.geo == g) & (gbd.sex == s) & (gbd.cause == c)].copy()
            # print(who_g.dtypes['year'])
            # print(gbd_g.dtypes['year'])
            if who_g.empty and gbd_g.empty:
                continue
            elif who_g.empty:
                to_concat.append(gbd_g)
            elif gbd_g.empty:
                to_concat.append(who_g)
            else:
                to_concat.append(trend_bridge(who_g, gbd_g))

In [42]:
res = pd.concat(to_concat, ignore_index=True, sort=False)

In [43]:
res[res.duplicated(subset=['geo', 'sex', 'cause', 'year'])]

,geo,sex,cause,year,value


In [44]:
res

,geo,sex,cause,year,value
0,sen,0,homicide,1990,1.610000
1,sen,0,homicide,1991,1.550000
2,sen,0,homicide,1992,1.510000
3,sen,0,homicide,1993,1.450000
4,sen,0,homicide,1994,1.460000
5,sen,0,homicide,1995,1.400000
6,sen,0,homicide,1996,1.370000
7,sen,0,homicide,1997,1.370000
8,sen,0,homicide,1998,1.360000
9,sen,0,homicide,1999,1.380000


In [45]:
res.query("geo == 'pan' and sex == 2 and cause == 'homicide'")

,geo,sex,cause,year,value
53652,pan,2,homicide,1984,1.647762
53653,pan,2,homicide,1985,1.519043
53654,pan,2,homicide,1986,2.114544
53655,pan,2,homicide,1987,2.454494
53656,pan,2,homicide,1988,2.259335
53657,pan,2,homicide,1989,2.503161
53658,pan,2,homicide,1990,2.670000
53659,pan,2,homicide,1991,2.770000
53660,pan,2,homicide,1992,2.850000
53661,pan,2,homicide,1993,2.900000


In [25]:
g = "pan" 
sex = 2 
c = "homicide"

who_g = who[(who.geo == g) & (who.sex == s) & (who.cause == c)].copy()
gbd_g = gbd[(gbd.geo == g) & (gbd.sex == s) & (gbd.cause == c)].copy()

In [64]:
who_g.dtypes

geo       object
sex        int64
cause     object
year       int64
value    float64
dtype: object

In [26]:
left_ = who_g.set_index('year').sort_index()['value']
right_ = gbd_g.set_index('year').sort_index()['value']

In [27]:
left_

year
1984    1.480923
1985    1.185365
1986    1.614027
1987    1.787137
1988    1.425139
1989    1.502126
1996    1.495986
1997    1.981115
1998    2.687893
1999    1.081187
2000    1.887910
2001    1.863158
2002    2.534529
2003    2.429605
2004    1.625193
2006    2.323033
Name: value, dtype: float64

In [28]:
right_

year
1990    2.67
1991    2.77
1992    2.85
1993    2.90
1994    2.94
1995    3.01
1996    2.92
1997    3.04
1998    3.01
1999    2.89
2000    2.94
2001    3.00
2002    3.09
2003    3.18
2004    3.20
2005    3.32
2006    3.41
2007    3.55
2008    3.71
2009    3.83
2010    3.76
2011    3.63
2012    3.54
2013    3.42
2014    3.38
2015    3.31
2016    3.22
Name: value, dtype: float64

In [29]:
bridge_length=10
bridge_point=1990

In [30]:
if bridge_point not in left_.index:
    before = left_.loc[:bridge_point].index[-1]
    if left_.index[-1] < bridge_point:
        bridge_point = before
    else:
        after = left_.loc[bridge_point:].index[0]
        if bridge_point - before >= after - bridge_point:
            bridge_point = after
        else:
            bridge_point = before

In [31]:
bridge_point

1989

In [33]:
if bridge_point < 1990:
    merged = pd.concat([left_.loc[:bridge_point], right_.loc[:]])

In [34]:
merged

year
1984    1.480923
1985    1.185365
1986    1.614027
1987    1.787137
1988    1.425139
1989    1.502126
1990    2.670000
1991    2.770000
1992    2.850000
1993    2.900000
1994    2.940000
1995    3.010000
1996    2.920000
1997    3.040000
1998    3.010000
1999    2.890000
2000    2.940000
2001    3.000000
2002    3.090000
2003    3.180000
2004    3.200000
2005    3.320000
2006    3.410000
2007    3.550000
2008    3.710000
2009    3.830000
2010    3.760000
2011    3.630000
2012    3.540000
2013    3.420000
2014    3.380000
2015    3.310000
2016    3.220000
Name: value, dtype: float64